In [52]:
import gspread
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import date
import pdfkit
config = pdfkit.configuration(wkhtmltopdf=r'C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe')

gc = gspread.oauth(credentials_filename='C:/Users/OCONNORB/AppData/Local/gspread/credentials.json')
sh = gc.open("Weather Station Visit Form")

In [53]:
station = 'Steph 8'
n = 5

# pull wx station visit sheet and put in dataframe
worksheet = sh.sheet1;
df = pd.DataFrame(worksheet.get_all_records())

# drop duplicates created by snow course (= multiple entries per visit) *unless we want snow course info?
# df.drop_duplicates(subset=['submissionid'], inplace=True, ignore_index=True)  # GOOGLE SHEETS
df.drop_duplicates(subset=['submissionid'], inplace=True, ignore_index=True)

# get entries for target station
df = df.loc[np.where(np.any(df == station, axis=1))]

# drop unwanted columns
# cols2keep = [6,16,17,18,19,20,21,22,23,24,25, 75, 76, 79, 134] # GOOGLE SHEETS
# cols2keep = [8, 19, 20,21,22,23,24,26,27,28,29,121,122,125, 183, 184] # CSV

cols2keep = ['Job_Start_Time', 
             'User',
             'What_jobs_are_being_completed_.Snow_Course',
             'What_jobs_are_being_completed_.Drone_Survey',
             'What_jobs_are_being_completed_.CF',
             'What_jobs_are_being_completed_.Sensor_Change',
             'What_jobs_are_being_completed_.Precip_Gage',
             'What_jobs_are_being_completed_.Lys_Calibration',
             'What_jobs_are_being_completed_.Tipping_Bucket_Calibration',
             'What_jobs_are_being_completed_.Data_Download',
             'What_jobs_are_being_completed_.General_Maintenance',
             'Sensor_Change.Type_of_Sensor',
             'Sensor_Change.Why_is_the_sensor_being_changed',
             'Sensor_Change.Additional_Notes',
             'General_Notes']

new_colnames = ['date', 'users', 'snow_course', 'drone', 'CF', 'sens_change', 'p_gage', 'lys_cal', 'buck_cal', 'data', 'gen_maint', 'sens_changed','reason', 'sens_notes', 'general_notes']

# df = df.iloc[:, cols2keep].set_axis(new_colnames, axis='columns')  # numerical indexing
df = df[cols2keep].set_axis(new_colnames, axis='columns')

# find the n most recent entries
df = df.sort_values(by="date")
df_table = df.iloc[-5:, :].copy()

# clean up for table
df_table[df_table == 'no'] = ' '
df_table[df_table == 'yes'] = 'Y'

display(df_table.style.set_caption(station + ' Pre-Trip Report for ' + str(date.today())))
df_table.to_html(open(station.replace(" ", "") + '_pretrip_example.html', 'w', encoding="utf-8"), index=False)

with open(station.replace(" ", "") + '_pretrip_example.html') as f:
    pdfkit.from_file(f, station.replace(" ", "") + '_pretrip_example.pdf',configuration=config)

,date,users,snow_course,drone,CF,sens_change,p_gage,lys_cal,buck_cal,data,gen_maint,sens_changed,reason,sens_notes,general_notes
118,2023-10-25 11:50:00,"AK, RB, JB",,,,,,,,Y,Y,,,,Taking down hobo station (old). No battery found in old hobo. Reconnected SR50 Brushed
109,2024-01-17 10:50:00,Ak sm,Y,,,,,,,Y,Y,,,,
137,2024-02-15 16:59:00,"AK, SM",,,,,,,,Y,Y,,,,Not enough snow for snow course :(
156,2024-04-12 15:29:00,LM BF AM,,,,Y,,,,Y,,SR50,Sensor was reading NAN,Swapped out 2 new sensors with same SDI address. Neither of them would work,Site was snow free
162,2024-04-18 16:06:00,"BF, AK, SM",,,,,,,,Y,Y,,,,SR50A with a cable was left in the logger box. The sdi address is 3. The sensor and cable were connected and tested but then unhooked and left in the box. The currently installed sr50a is not functioning and has been unplugged from power.
